In [ ]:
%pip install python-dotenv pandas sepaxml

In [ ]:
import os
from dotenv import load_dotenv
from dotenv import dotenv_values

# Load the .env file
CONFIG = dotenv_values(".env")

In [ ]:
QUARTAL = "Abr_YQ-2025-2"
IMPORT_FILE = "./EEG-Faktura Import/abrechnung_Abr_YQ-2025-2_export.xlsx"
#IMPORT_FILE="/home/martin/Downloads/RC101533_abrechnung_Abr_YQ-2025-2_export (2).xlsx"

In [ ]:
NAME = CONFIG["NAME"]
IBAN = CONFIG["IBAN"]
BIC = CONFIG["BIC"]
CREDITOR_ID = CONFIG["CREDITOR_ID"]
ADDRESS = {
    # The address and all of its fields are optional but in some countries they are required
    "address_type": "ADDR",  # valid: ADDR, PBOX, HOME, BIZZ, MLTO, DLVY
    "department": "Head Office",
    "subdepartment": None,
    "street_name": CONFIG["STREET_NAME"],
    "building_number": CONFIG["BUILDING_NUMBER"],
    "postcode": CONFIG["POSTCODE"],
    "town": CONFIG["TOWN"],
    "country": CONFIG["COUNTRY"],
    "country_subdivision": None,
    "lines": ["Line 1", "Line 2"],
}

In [ ]:
import pandas as pd

df = pd.read_excel(IMPORT_FILE, parse_dates=['Datum', 'Empfänger Mandatsausstellung'])

SEPA Rechnungen Abbucher

In [ ]:
# prüfe umsatz mit EEGfaktura Dashboard

sum = 0.0

for m_id in df['Empfänger Mitgliedsnummer'].unique():
    m_df = df[df['Empfänger Mitgliedsnummer'] == m_id]
    m_df_re = m_df[m_df['Dokumenttyp'] == "Rechnung"]
    m_df_gs = m_df[m_df['Dokumenttyp'] == "Gutschrift"]
    # mitgliedsbeitrag abziehen    
    soll = m_df_re['Rechnungsbetrag Brutto'].sum() - 5.0
    haben = m_df_gs['Rechnungsbetrag Brutto'].sum()
    #print(soll, haben)
    sum = sum + soll + haben
    
    
sum / 2
    
# soll den wert des Umsatzes ausgeben

In [ ]:
import datetime, uuid
import math


from sepaxml import SepaDD

config = {
    "name": NAME,
    "IBAN": IBAN,
    "BIC": BIC,
    "batch": True,
    "creditor_id": CREDITOR_ID,  # supplied by your bank or financial authority
    "currency": "EUR",  # ISO 4217
    # "instrument": "B2B",  # - default is CORE (B2C)
    "address": ADDRESS
}
sepa_invoices = SepaDD(config, schema="pain.008.001.02", clean=True)

accounting = df[df['Abrechnung'] == QUARTAL]
invoices = accounting[accounting['Dokumenttyp'] == "Rechnung"]
invoices
invoices = invoices[[
    'Nummer', 
    'Empfänger Kontoeigner', 
    'Empfänger Konto IBAN', 
    'Rechnungsbetrag Brutto', 
    'Empfänger Mitgliedsnummer',
    'Empfänger Mandatsausstellung'
]]

invoices['BIC'] = 'NOTPROVIDED'

invoices = invoices.rename(columns={
    'Nummer': 'description',
    'Empfänger Kontoeigner': 'name',
    'Empfänger Konto IBAN': 'IBAN',
    'Rechnungsbetrag Brutto': 'amount',
    'Empfänger Mitgliedsnummer': 'mandate_id',
    'Empfänger Mandatsausstellung': 'mandate_date'
})

# remove spaces
invoices['IBAN'] = invoices['IBAN'].apply(lambda x: x.replace(" ", ""))

# values in CENT!
invoices['amount'] = invoices['amount'] * 100
invoices['amount'] = invoices['amount'].apply(round)
invoices['collection_date'] = datetime.date.today()
invoices['mandate_date'] = invoices['mandate_date'].apply(lambda x: x.to_pydatetime().date())
invoices['mandate_id'] = invoices['mandate_id'].apply(lambda x: str(x).zfill(3))

invoices['type'] = "RCUR"


for index, row in invoices.iterrows():
    payment = row.to_dict()
    #print(payment)
    sepa_invoices.add_payment(payment)


xml_file = f"XML Lastschriften/{QUARTAL}.xml"

with open(xml_file, 'wb') as file:
    file.write(sepa_invoices.export(validate=True))
    
print(xml_file)


SEPA Gutschriften

In [ ]:
from sepaxml import SepaTransfer


config = {
    "name": NAME,
    "IBAN": IBAN,
    "BIC": BIC,
    "batch": True,
    "currency": "EUR",  # ISO 4217
    "address": ADDRESS
}

sepa_credits = SepaTransfer(config, clean=True)


accounting = df[df['Abrechnung'] == QUARTAL]
credits = accounting[accounting['Dokumenttyp'] == "Gutschrift"]
credits = credits[[
    'Nummer', 
    'Empfänger Kontoeigner', 
    'Empfänger Konto IBAN', 
    'Rechnungsbetrag Brutto', 
]]
credits['BIC'] = 'NOTPROVIDED'
credits = credits.rename(columns={
    'Nummer': 'description',
    'Empfänger Kontoeigner': 'name',
    'Empfänger Konto IBAN': 'IBAN',
    'Rechnungsbetrag Brutto': 'amount',
})

credits['IBAN'] = credits['IBAN'].apply(lambda x: x.replace(" ", ""))

# values in CENT!
credits['amount'] = credits['amount'] * 100
credits['amount'] = credits['amount'].apply(math.floor)
credits['execution_date'] = datetime.date.today() + datetime.timedelta(days=2)

for index, row in credits.iterrows():
    payment = row.to_dict()
    sepa_credits.add_payment(payment)


xml_file = f"XML Gutschriften/{QUARTAL}.xml"

with open(xml_file, 'wb') as file:
    file.write(sepa_credits.export(validate=True))
    
print(xml_file)
